In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, accuracy_score, f1_score
from imblearn.over_sampling import SMOTE
from sklearn.impute import SimpleImputer
import joblib

In [2]:
def load_and_preprocess_data(file_path, is_train=True):
    data = pd.read_excel(file_path)
    
    # Drop unnecessary columns
    data = data.drop(columns=['id', 'pmonths', 'profit', 'pastEmail'], errors='ignore')

    # Fill missing values
    data['custAge'] = data['custAge'].fillna(data['custAge'].median())
    for col in ['profession', 'marital', 'schooling', 'day_of_week']:
        data[col] = data[col].fillna('Unknown')

    # Convert categorical binary variables
    binary_cols = ['default', 'housing', 'loan']
    for col in binary_cols:
        data[col] = data[col].map({'yes': 1, 'no': 0, 'unknown': 0})

    # Handle `pdays`
    data['pdays_no_contact'] = (data['pdays'] == 999).astype(int)
    data['pdays'] = data['pdays'].replace(999, -1)
    
    if is_train:
        # Convert target variable
        data['responded'] = data['responded'].apply(lambda x: 1 if x == 'yes' else 0)
        X = data.drop('responded', axis=1)
        y = data['responded']
        return X, y
    else:
        X = data
        return X

In [3]:
# Load and preprocess training data
X, y = load_and_preprocess_data("/Users/vijaykulkarni/Downloads/Propensify/train.xlsx", is_train=True)
X_test_final = load_and_preprocess_data("/Users/vijaykulkarni/Downloads/Propensify/test.xlsx", is_train=False)

In [4]:
# Split the data into training and validation sets
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)


In [5]:
# Define feature processing
categorical_cols = ['profession', 'marital', 'schooling', 'contact', 'month', 'day_of_week', 'poutcome']
numerical_cols = X_train.select_dtypes(include=['float64', 'int64']).columns

numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

In [6]:
# Process training and validation data
X_train_processed = preprocessor.fit_transform(X_train)
X_valid_processed = preprocessor.transform(X_valid)

In [7]:
# Handle imbalanced data using SMOTE on training data
smote = SMOTE(random_state=42)
X_train_bal, y_train_bal = smote.fit_resample(X_train_processed, y_train)

In [8]:
# Process test data
X_test_processed = preprocessor.transform(X_test_final)

In [9]:
# Define classifiers and parameter grids
classifiers = {
    'Random Forest': RandomForestClassifier(random_state=42),
    'Gradient Boosting': GradientBoostingClassifier(random_state=42),
    'AdaBoost': AdaBoostClassifier(random_state=42),
    'Logistic Regression': LogisticRegression(random_state=42, max_iter=500),
    'XGBoost': XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='logloss')
}

In [10]:
param_grids = {
    'Random Forest': {
        'n_estimators': [100, 200, 300],
        'max_depth': [None, 10, 20],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4]
    },
    'Gradient Boosting': {
        'n_estimators': [100, 200],
        'max_depth': [3, 5],
        'learning_rate': [0.01, 0.1],
        'subsample': [0.8, 1.0]
    },
    'AdaBoost': {
        'n_estimators': [50, 100],
        'learning_rate': [0.01, 0.1]
    },
    'XGBoost': {
        'n_estimators': [100, 200],
        'max_depth': [3, 5, 10],
        'learning_rate': [0.01, 0.1],
        'colsample_bytree': [0.8, 1.0]
    }
}


In [11]:
# Train and evaluate models
best_model = None
best_f1 = 0
best_accuracy = 0

# Store models with accuracy > 85%
models_with_high_accuracy = []

for name, model in classifiers.items():
    print(f"Evaluating {name}...")
    
    try:
        # If hyperparameters are available for the model, apply GridSearchCV
        if name in param_grids:
            param_grid = param_grids[name]
            grid_search = GridSearchCV(model, param_grid, cv=5, scoring='f1', n_jobs=-1)
            grid_search.fit(X_train_bal, y_train_bal)
            model = grid_search.best_estimator_
        
        # Cross-Validation for each model using F1 score and accuracy
        cv_f1 = cross_val_score(model, X_train_bal, y_train_bal, cv=5, scoring='f1').mean()
        cv_accuracy = cross_val_score(model, X_train_bal, y_train_bal, cv=5, scoring='accuracy').mean()
        
        print(f"{name} Cross-Validation Results:")
        print(f"F1 Score (CV): {cv_f1}")
        print(f"Accuracy (CV): {cv_accuracy}")

        # Evaluate on Validation Data
        model.fit(X_train_bal, y_train_bal)
        y_valid_pred = model.predict(X_valid_processed)
        accuracy = accuracy_score(y_valid, y_valid_pred)
        f1 = f1_score(y_valid, y_valid_pred, pos_label=1)
        
        # Print performance
        print(f"{name} Validation Results:")
        print(f"Validation Accuracy: {accuracy}")
        print(f"Validation F1 Score (Class 1): {f1}")
        print("Classification Report:\n", classification_report(y_valid, y_valid_pred))
        
        # Store models with accuracy > 85%
        if accuracy > 0.85:
            models_with_high_accuracy.append((model, f1, accuracy, name))
            if f1 > best_f1:
                best_f1 = f1
                best_accuracy = accuracy
                best_model = model

    except Exception as e:
        print(f"Error with model {name}: {e}")


Evaluating Random Forest...
Random Forest Cross-Validation Results:
F1 Score (CV): 0.938194037207583
Accuracy (CV): 0.944504224312628
Random Forest Validation Results:
Validation Accuracy: 0.8883495145631068
Validation F1 Score (Class 1): 0.4140127388535032
Classification Report:
               precision    recall  f1-score   support

           0       0.92      0.95      0.94      1465
           1       0.50      0.36      0.41       183

    accuracy                           0.89      1648
   macro avg       0.71      0.66      0.68      1648
weighted avg       0.87      0.89      0.88      1648

Evaluating Gradient Boosting...
Gradient Boosting Cross-Validation Results:
F1 Score (CV): 0.9048125230599888
Accuracy (CV): 0.9232981785771888
Gradient Boosting Validation Results:
Validation Accuracy: 0.8907766990291263
Validation F1 Score (Class 1): 0.4117647058823529
Classification Report:
               precision    recall  f1-score   support

           0       0.92      0.96      0

/Users/vijaykulkarni/anaconda3/lib/python3.11/site-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
/Users/vijaykulkarni/anaconda3/lib/python3.11/site-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
/Users/vijaykulkarni/anaconda3/lib/python3.11/site-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
/Users/vijaykulkarni/anaconda3/lib/python3.11/site-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME

AdaBoost Cross-Validation Results:
F1 Score (CV): 0.785612798407466
Accuracy (CV): 0.8007541625964464


/Users/vijaykulkarni/anaconda3/lib/python3.11/site-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


AdaBoost Validation Results:
Validation Accuracy: 0.8349514563106796
Validation F1 Score (Class 1): 0.4448979591836735
Classification Report:
               precision    recall  f1-score   support

           0       0.94      0.86      0.90      1465
           1       0.36      0.60      0.44       183

    accuracy                           0.83      1648
   macro avg       0.65      0.73      0.67      1648
weighted avg       0.88      0.83      0.85      1648

Evaluating Logistic Regression...
Logistic Regression Cross-Validation Results:
F1 Score (CV): 0.7297554252298706
Accuracy (CV): 0.7491876687594701
Logistic Regression Validation Results:
Validation Accuracy: 0.7961165048543689
Validation F1 Score (Class 1): 0.4186851211072664
Classification Report:
               precision    recall  f1-score   support

           0       0.95      0.81      0.88      1465
           1       0.31      0.66      0.42       183

    accuracy                           0.80      1648
   macro a

/Users/vijaykulkarni/anaconda3/lib/python3.11/site-packages/xgboost/core.py:158: UserWarning: [19:27:35] WARNING: /var/folders/c_/qfmhj66j0tn016nkx_th4hxm0000gp/T/abs_b6qk1lz_ug/croot/xgboost-split_1724073748391/work/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/vijaykulkarni/anaconda3/lib/python3.11/site-packages/xgboost/core.py:158: UserWarning: [19:27:35] WARNING: /var/folders/c_/qfmhj66j0tn016nkx_th4hxm0000gp/T/abs_b6qk1lz_ug/croot/xgboost-split_1724073748391/work/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/vijaykulkarni/anaconda3/lib/python3.11/site-packages/xgboost/core.py:158: UserWarning: [19:27:35] WARNING: /var/folders/c_/qfmhj66j0tn016nkx_th4hxm0000gp/T/abs_b6qk1lz_ug/croot/xgboost-split_1724073748391/work/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/vijaykulkarni/anacond

XGBoost Cross-Validation Results:
F1 Score (CV): 0.9245559441144341
Accuracy (CV): 0.9323615518611588


/Users/vijaykulkarni/anaconda3/lib/python3.11/site-packages/xgboost/core.py:158: UserWarning: [19:28:15] WARNING: /var/folders/c_/qfmhj66j0tn016nkx_th4hxm0000gp/T/abs_b6qk1lz_ug/croot/xgboost-split_1724073748391/work/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


XGBoost Validation Results:
Validation Accuracy: 0.8774271844660194
Validation F1 Score (Class 1): 0.4127906976744186
Classification Report:
               precision    recall  f1-score   support

           0       0.92      0.94      0.93      1465
           1       0.44      0.39      0.41       183

    accuracy                           0.88      1648
   macro avg       0.68      0.66      0.67      1648
weighted avg       0.87      0.88      0.87      1648



In [12]:
# Select the best model from those with accuracy > 85%
if best_model:
    print(f"Best model is: {best_model.__class__.__name__} with F1 score: {best_f1} and accuracy: {best_accuracy}")

    # Save the best model for future use
    joblib.dump(best_model, 'best_model.pkl')

    # Apply the best model to the test data (without labels)
    try:
        final_predictions = best_model.predict(X_test_processed)

        # Prepare data for output
        test_data_with_predictions = pd.concat([
            X_test_final.reset_index(drop=True),
            pd.Series(final_predictions, name='predicted_responded')
        ], axis=1)
        
        # Save to Excel
        test_data_with_predictions.to_excel('/Users/vijaykulkarni/Downloads/Propensify/test_predictions.xlsx', index=False)
        print("Test predictions saved to '/Users/vijaykulkarni/Downloads/Propensify/test_predictions.xlsx'")

    except Exception as e:
        print(f"Error with final test predictions: {e}")
else:
    print("No model met the accuracy threshold of >85%.")

Best model is: RandomForestClassifier with F1 score: 0.4140127388535032 and accuracy: 0.8883495145631068
Test predictions saved to '/Users/vijaykulkarni/Downloads/Propensify/test_predictions.xlsx'
